In [1]:
# Install necessary packages
!pip install boto3 sagemaker

In [4]:
import pandas as pd
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.estimator import Estimator

In [5]:
# Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::241533149150:role/aws-sagemaker-role


In [6]:
# S3 bucket details
s3_bucket = "laboratory-diagnostics-data"
s3_prefix = "SourceData/diabeticprediction"
file_path = f"s3://{s3_bucket}/{s3_prefix}/diabetes_prediction.csv"
print(file_path)

s3://laboratory-diagnostics-data/SourceData/diabeticprediction/diabetes_prediction.csv


In [7]:
pip install --upgrade s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.63
    Uninstalling botocore-1.35.63:
      Successfully uninstalled botocore-1.35.63
  Attempting uninstall: s3fs
    Found existing installation: s3fs 0.4.2
    Uninstalling s3fs-0.4.2:
      Successfully uninstalled s3fs-0.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.36.4 requires botocore==1.35.63, but you have botocore 1.35.36 which is incompatible.
boto3 1.35.63 requires botocore<1.36.0,>=1.35.63, but you have botocore 1.35.36 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Read data directly from S3
df = pd.read_csv(file_path)
print(df.head())

   gender  hypertension  heart_disease smoking_history  hbA1c_level  \
0    Male             1              0  Current smoker          5.7   
1  Female             0              1   Former smoker          6.1   
2  Female             1              1    Never smoked          5.8   
3    Male             0              0  Current smoker          6.3   
4  Female             1              0   Former smoker          5.9   

   pregnancies  glucose  blood_pressure  skin_thickness  insulin   bmi  \
0            0      110              80              32     15.0  28.0   
1            1      150              90              20     20.5  31.2   
2            2      130              85              22     10.8  25.5   
3            0      180             100              25     12.4  29.1   
4            3      140              75              18     14.3  26.4   

   diabetes_pedigree_function  age  is_diabetic  
0                         0.6   45            1  
1                         0.

In [8]:
# Ensure labels are binary (0 or 1)
df['is_diabetic'] = df['is_diabetic'].apply(lambda x: 1 if x > 0 else 0)

In [9]:
# Data Preprocessing and Feature Engineering
# Encode categorical variables (gender and smoking_history) using one-hot encoding
df = pd.get_dummies(df, columns=['gender', 'smoking_history'], drop_first=True)

In [10]:
print(df.head())

   hypertension  heart_disease  hbA1c_level  pregnancies  glucose  \
0             1              0          5.7            0      110   
1             0              1          6.1            1      150   
2             1              1          5.8            2      130   
3             0              0          6.3            0      180   
4             1              0          5.9            3      140   

   blood_pressure  skin_thickness  insulin   bmi  diabetes_pedigree_function  \
0              80              32     15.0  28.0                         0.6   
1              90              20     20.5  31.2                         0.7   
2              85              22     10.8  25.5                         0.8   
3             100              25     12.4  29.1                         0.9   
4              75              18     14.3  26.4                         0.6   

   age  is_diabetic  gender_Male  smoking_history_Former smoker  \
0   45            1            1     

In [11]:
# Split data into features and labels
X = df.drop('is_diabetic', axis=1)
y = df['is_diabetic']

In [12]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Save preprocessed data to S3
train_data = pd.DataFrame(X_train)
train_data['is_diabetic'] = y_train.values
test_data = pd.DataFrame(X_test)
test_data['is_diabetic'] = y_test.values

In [14]:
train_data.to_csv('train.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)

In [15]:
train_s3_path = f's3://{sagemaker_session.default_bucket()}/diabetes-prediction/train/train.csv'
test_s3_path = f's3://{sagemaker_session.default_bucket()}/diabetes-prediction/test/test.csv'

print(train_s3_path)
print(test_s3_path)

s3://sagemaker-us-east-2-241533149150/diabetes-prediction/train/train.csv
s3://sagemaker-us-east-2-241533149150/diabetes-prediction/test/test.csv


In [16]:
# Upload training and testing data to S3
s3_client = boto3.client('s3')
s3_client.upload_file('train.csv', sagemaker_session.default_bucket(), 'diabetes-prediction/train/train.csv')
s3_client.upload_file('test.csv', sagemaker_session.default_bucket(), 'diabetes-prediction/test/test.csv')


In [17]:
# Set up and configure XGBoost model
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")

In [18]:
xgb = Estimator(
    image_uri=xgboost_container,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f's3://{sagemaker_session.default_bucket()}/diabetes-prediction/output',
    role=role,
    sagemaker_session=sagemaker_session
)

In [19]:
# Hyperparameters for XGBoost
xgb.set_hyperparameters(
    objective="binary:logistic",
    num_round=10,
    max_depth=3,
    eta=0.2,
    subsample=0.8,
    colsample_bytree=0.8
)

In [20]:
# Prepare data inputs for training
train_input = TrainingInput(s3_data=train_s3_path, content_type="csv")

In [21]:
# Train the model
xgb.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-11-29-03-12-30-012


2024-11-29 03:12:31 Starting - Starting the training job...
2024-11-29 03:12:46 Starting - Preparing the instances for training...
2024-11-29 03:13:30 Downloading - Downloading the training image......
2024-11-29 03:14:31 Training - Training image download completed. Training in progress.
2024-11-29 03:14:31 Uploading - Uploading generated training model[2024-11-29 03:14:26.537 ip-10-0-204-208.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[2024-11-29 03:14:26

In [22]:
# Deploy the model to an endpoint
predictor = xgb.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-11-29-03-15-17-038
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-11-29-03-15-17-038
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-11-29-03-15-17-038


-----!

In [23]:
# Transform test set for predictions
import io
from sklearn.metrics import accuracy_score

In [24]:
test_data_df = pd.DataFrame(X_test, columns=X.columns)
# Convert the DataFrame to a CSV string
csv_buffer = io.StringIO()
test_data_df.to_csv(csv_buffer, header=False, index=False)

In [25]:
print(test_data_df.head())

    hypertension  heart_disease  hbA1c_level  pregnancies  glucose  \
0              1              0          5.7            0      110   
17             0              1          6.2            1      140   
15             0              0          6.1            0      155   
1              0              1          6.1            1      150   

    blood_pressure  skin_thickness  insulin   bmi  diabetes_pedigree_function  \
0               80              32     15.0  28.0                         0.6   
17              90              23     18.8  28.4                         0.6   
15              93              25     13.1  29.5                         0.7   
1               90              20     20.5  31.2                         0.7   

    age  gender_Male  smoking_history_Former smoker  \
0    45            1                              0   
17   50            1                              0   
15   48            1                              0   
1    52            0   

In [26]:
# Encode the CSV string to bytes
csv_bytes = csv_buffer.getvalue().encode('utf-8')

# Make predictions with specified content type
predictions = predictor.predict(csv_bytes, initial_args={'ContentType': 'text/csv'})

# Threshold predictions to get binary outcomes
predictions = [float(pred) for pred in predictions.decode('utf-8').strip().split(',')]
predictions = (pd.Series(predictions) > 0.4).astype(int)


# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 100.00%
